In [1]:
import pandas as pd
import numpy as np

In [2]:
import nltk

In [12]:
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines=True)

In [5]:
type(data)

pandas.core.frame.DataFrame

In [6]:
data.head(5)

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [7]:
data.tail(5)

,article_link,headline,is_sarcastic
26704,https://www.huffingtonpost.com/entry/american-...,american politics in moral free-fall,0
26705,https://www.huffingtonpost.com/entry/americas-...,america's best 20 hikes,0
26706,https://www.huffingtonpost.com/entry/reparatio...,reparations and obama,0
26707,https://www.huffingtonpost.com/entry/israeli-b...,israeli ban targeting boycott supporters raise...,0
26708,https://www.huffingtonpost.com/entry/gourmet-g...,gourmet gifts for the foodie 2014,0


In [8]:
data.is_sarcastic.value_counts()

0    14985
1    11724
Name: is_sarcastic, dtype: int64

In [10]:
data = data.drop('article_link', axis=1)

In [11]:
data.head(5)

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [18]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import gensim.downloader

model = gensim.downloader.load('glove-twitter-25')

[=================================================-] 98.8% 103.5/104.8MB downloaded


In [54]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

nltk.download('stopwords')
nltk.download('punkt')

def tokenize_headline(sentence):
    # Tokenize the sentence
    tokens = word_tokenize(sentence.lower())

    # Remove stopwords and punctuations
    stopwords_set = set(stopwords.words('english'))
    tokens = [re.sub(r'[^a-zA-Z\-]', '', token) for token in tokens if token not in stopwords_set and token not in string.punctuation]
    tokens = [subtoken for token in tokens for subtoken in token.split('-')]
    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [158]:
tokenize_headline('j.k. rowling, announced he is stepping out of the Office! Can you beleive it?')

['jk', 'rowling', 'announced', 'stepping', 'office', 'beleive']

In [56]:
data['tokenized_headline'] = data['headline'].apply(tokenize_headline)

In [57]:
data.head(5)

,headline,is_sarcastic,tokenized_headline
0,former versace store clerk sues over secret 'b...,0,"[former, versace, store, clerk, sues, secret, ..."
1,the 'roseanne' revival catches up to our thorn...,0,"[roseanne, revival, catches, thorny, political..."
2,mom starting to fear son's web series closest ...,1,"[mom, starting, fear, son, s, web, series, clo..."
3,"boehner just wants wife to listen, not come up...",1,"[boehner, wants, wife, listen, come, alternati..."
4,j.k. rowling wishes snape happy birthday in th...,0,"[jk, rowling, wishes, snape, happy, birthday, ..."


In [75]:
from gensim.models import Word2Vec

In [78]:
vocab = data['tokenized_headline'].tolist()

In [79]:
len(vocab)

26708

In [96]:
model = Word2Vec(vocab, vector_size=100, window=5, min_count=1, workers=4)

In [97]:
model.train(vocab, total_examples=len(vocab), epochs=10)

(1905210, 2005780)

In [102]:
def vectorize_sentence(sent):
  return [model.wv[x] for x in sent]

In [103]:
data['vectorized_headline'] = data['tokenized_headline'].apply(vectorize_sentence)

In [104]:
data.head(5)

,headline,is_sarcastic,tokenized_headline,vectorized_headline
0,former versace store clerk sues over secret 'b...,0,"[former, versace, store, clerk, sues, secret, ...","[[-0.5125088, 0.8833395, 0.26111025, -0.270109..."
1,the 'roseanne' revival catches up to our thorn...,0,"[roseanne, revival, catches, thorny, political...","[[-0.09110452, 0.13334975, 0.037916254, 0.0042..."
2,mom starting to fear son's web series closest ...,1,"[mom, starting, fear, son, s, web, series, clo...","[[-0.28869796, 0.58083326, 0.70630294, 0.33058..."
3,"boehner just wants wife to listen, not come up...",1,"[boehner, wants, wife, listen, come, alternati...","[[-0.28714094, 0.38520864, -0.113378726, -0.44..."
4,j.k. rowling wishes snape happy birthday in th...,0,"[jk, rowling, wishes, snape, happy, birthday, ...","[[-0.08106669, 0.15479226, 0.054842737, -0.016..."


In [201]:
data.drop(5322, inplace=True)

In [107]:
max_len = 0
for _, row in data.iterrows():
  if len(row['tokenized_headline']) > max_len:
    max_len = len(row['tokenized_headline'])
    
max_len

27

In [159]:
import torch

def pad_sentence(sentence, max_len, dimension):
  padded = torch.zeros(max_len, dimension)
  padded[:sentence.size(0)] = sentence
  return padded

In [162]:
len(pad_sentence(tensorized_sentences[0], 27, 100))

27

In [204]:
data['tensorized_headline'] = [pad_sentence(torch.tensor(x), max_len, 100) for x in data['vectorized_headline']]

In [205]:
data.head(5)

,headline,is_sarcastic,tokenized_headline,vectorized_headline,tensorized_headline
0,former versace store clerk sues over secret 'b...,0,"[former, versace, store, clerk, sues, secret, ...","[[-0.5125088, 0.8833395, 0.26111025, -0.270109...","[[tensor(-0.5125), tensor(0.8833), tensor(0.26..."
1,the 'roseanne' revival catches up to our thorn...,0,"[roseanne, revival, catches, thorny, political...","[[-0.09110452, 0.13334975, 0.037916254, 0.0042...","[[tensor(-0.0911), tensor(0.1333), tensor(0.03..."
2,mom starting to fear son's web series closest ...,1,"[mom, starting, fear, son, s, web, series, clo...","[[-0.28869796, 0.58083326, 0.70630294, 0.33058...","[[tensor(-0.2887), tensor(0.5808), tensor(0.70..."
3,"boehner just wants wife to listen, not come up...",1,"[boehner, wants, wife, listen, come, alternati...","[[-0.28714094, 0.38520864, -0.113378726, -0.44...","[[tensor(-0.2871), tensor(0.3852), tensor(-0.1..."
4,j.k. rowling wishes snape happy birthday in th...,0,"[jk, rowling, wishes, snape, happy, birthday, ...","[[-0.08106669, 0.15479226, 0.054842737, -0.016...","[[tensor(-0.0811), tensor(0.1548), tensor(0.05..."


In [255]:
train, test = train_test_split(data, test_size=0.2)

In [256]:
train.head(5)

,headline,is_sarcastic,tokenized_headline,vectorized_headline,tensorized_headline
21605,iraq is investigating alleged executions of su...,0,"[iraq, investigating, alleged, executions, sun...","[[-0.32991463, 0.45350707, 0.13634884, -0.3666...","[[tensor(-0.3299), tensor(0.4535), tensor(0.13..."
11011,lena dunham dings woody allen,0,"[lena, dunham, dings, woody, allen]","[[-0.29160574, 0.4219882, 0.119682744, -0.0191...","[[tensor(-0.2916), tensor(0.4220), tensor(0.11..."
6473,trump launches another sexist tweet in newest ...,0,"[trump, launches, another, sexist, tweet, newe...","[[-0.16637842, 0.37916443, -1.2198406, -1.3125...","[[tensor(-0.1664), tensor(0.3792), tensor(-1.2..."
25289,high school suspends hunky student for wearing...,1,"[high, school, suspends, hunky, student, weari...","[[-0.31187633, 1.2827308, 0.7460096, 0.4311858...","[[tensor(-0.3119), tensor(1.2827), tensor(0.74..."
13762,smiling willie nelson reflects on a lifetime o...,1,"[smiling, willie, nelson, reflects, lifetime, ...","[[-0.17280853, 0.22684434, 0.14137684, -0.0285...","[[tensor(-0.1728), tensor(0.2268), tensor(0.14..."


In [257]:
test.head(5)

,headline,is_sarcastic,tokenized_headline,vectorized_headline,tensorized_headline
23775,this innocent map looks just like a penis,0,"[innocent, map, looks, like, penis]","[[-0.14850184, 0.23281741, 0.2600559, -0.00010...","[[tensor(-0.1485), tensor(0.2328), tensor(0.26..."
24602,alligator and python locked in death duel on g...,0,"[alligator, python, locked, death, duel, golf,...","[[-0.06328432, 0.0735361, 0.058583416, -0.0463...","[[tensor(-0.0633), tensor(0.0735), tensor(0.05..."
25610,god recalls life-changing encounter with 8-yea...,1,"[god, recalls, life, changing, encounter, , ye...","[[-0.41928425, 0.58477724, 0.4211417, 0.189872...","[[tensor(-0.4193), tensor(0.5848), tensor(0.42..."
12917,"amtrak train derails in vermont, seven taken t...",0,"[amtrak, train, derails, vermont, seven, taken...","[[-0.069038644, 0.10970071, 0.055921633, -0.03...","[[tensor(-0.0690), tensor(0.1097), tensor(0.05..."
14423,8th grader caked in makeup probably really con...,1,"[th, grader, caked, makeup, probably, really, ...","[[-0.5196069, 1.0023136, 0.6830433, 0.783615, ...","[[tensor(-0.5196), tensor(1.0023), tensor(0.68..."


In [209]:
import torch.nn as nn

linear_model = nn.Linear(max_len, 2)

In [250]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = train['tensorized_headline'].tolist()
        self.y = train['is_sarcastic'].tolist()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out


In [251]:
input_size = 100
num_classes = 2
num_epochs = 5
batch_size = 64


model_lr = LogisticRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_lr.parameters(), lr=0.01)

In [258]:
train

,headline,is_sarcastic,tokenized_headline,vectorized_headline,tensorized_headline
21605,iraq is investigating alleged executions of su...,0,"[iraq, investigating, alleged, executions, sun...","[[-0.32991463, 0.45350707, 0.13634884, -0.3666...","[[tensor(-0.3299), tensor(0.4535), tensor(0.13..."
11011,lena dunham dings woody allen,0,"[lena, dunham, dings, woody, allen]","[[-0.29160574, 0.4219882, 0.119682744, -0.0191...","[[tensor(-0.2916), tensor(0.4220), tensor(0.11..."
6473,trump launches another sexist tweet in newest ...,0,"[trump, launches, another, sexist, tweet, newe...","[[-0.16637842, 0.37916443, -1.2198406, -1.3125...","[[tensor(-0.1664), tensor(0.3792), tensor(-1.2..."
25289,high school suspends hunky student for wearing...,1,"[high, school, suspends, hunky, student, weari...","[[-0.31187633, 1.2827308, 0.7460096, 0.4311858...","[[tensor(-0.3119), tensor(1.2827), tensor(0.74..."
13762,smiling willie nelson reflects on a lifetime o...,1,"[smiling, willie, nelson, reflects, lifetime, ...","[[-0.17280853, 0.22684434, 0.14137684, -0.0285...","[[tensor(-0.1728), tensor(0.2268), tensor(0.14..."
...,...,...,...,...,...
9138,trash bag taped over broken southwest plane wi...,1,"[trash, bag, taped, broken, southwest, plane, ...","[[-0.31330737, 0.4112902, 0.23405984, 0.122848...","[[tensor(-0.3133), tensor(0.4113), tensor(0.23..."
26542,"4 copy editors killed in ongoing ap style, chi...",1,"[, copy, editors, killed, ongoing, ap, style, ...","[[-0.43619215, 0.32279488, 1.4544721, 2.348317...","[[tensor(-0.4362), tensor(0.3228), tensor(1.45..."
13970,man in solitary confinement can't break with r...,1,"[man, solitary, confinement, ca, nt, break, re...","[[-0.88942754, 0.8432814, -0.17420548, 0.55902...","[[tensor(-0.8894), tensor(0.8433), tensor(-0.1..."
21013,local news anchor mistakenly reveals salary du...,1,"[local, news, anchor, mistakenly, reveals, sal...","[[-0.35636067, 0.73983884, 1.009919, -0.320832...","[[tensor(-0.3564), tensor(0.7398), tensor(1.00..."


In [259]:
num_splits = 5
for split in range(num_splits):
    # Create train and validation sets for the current split
    train, validation = train_test_split(train, test_size = 0.1) 
    train_dataset = CustomDataset(train['tensorized_headline'], train['is_sarcastic'])
    val_dataset = CustomDataset(validation['tensorized_headline'], validation['is_sarcastic'])

    # Create data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Train the model
    for epoch in range(num_epochs):
        model_lr.train()
        for inputs, targets in train_loader:
            # Forward pass
            outputs = model_lr(inputs)
            target_one_hot = F.one_hot(targets, 2)
            loss = criterion(outputs, target_one_hot)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validate the model
        model_lr.eval()
        with torch.no_grad():
            val_loss = 0.0
            total_correct = 0
            total_samples = 0
            for inputs, targets in val_loader:
                outputs = model_lr(inputs)
                target_one_hot = F.one_hot(targets, 2)
                val_loss += criterion(outputs, target_one_hot).item()

                _, predicted = torch.max(outputs, dim=1)
                total_correct += (predicted == target_one_hot).sum().item()
                total_samples += targets.size(0)

            val_loss /= len(val_loader)
            accuracy = total_correct / total_samples

        
        print(f'Split [{split+1}/{num_splits}], Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

Split [1/5], Epoch [1/5], Val Loss: 2.5236, Accuracy: 0.4282
Split [1/5], Epoch [2/5], Val Loss: 2.5096, Accuracy: 0.4452
Split [1/5], Epoch [3/5], Val Loss: 2.5055, Accuracy: 0.4452
Split [1/5], Epoch [4/5], Val Loss: 2.5031, Accuracy: 0.4461
Split [1/5], Epoch [5/5], Val Loss: 2.5014, Accuracy: 0.4463
Split [2/5], Epoch [1/5], Val Loss: 2.5005, Accuracy: 0.4515
Split [2/5], Epoch [2/5], Val Loss: 2.4993, Accuracy: 0.4525
Split [2/5], Epoch [3/5], Val Loss: 2.4984, Accuracy: 0.4535
Split [2/5], Epoch [4/5], Val Loss: 2.4976, Accuracy: 0.4537
Split [2/5], Epoch [5/5], Val Loss: 2.4970, Accuracy: 0.4536
Split [3/5], Epoch [1/5], Val Loss: 2.5014, Accuracy: 0.4502
Split [3/5], Epoch [2/5], Val Loss: 2.5008, Accuracy: 0.4520
Split [3/5], Epoch [3/5], Val Loss: 2.5004, Accuracy: 0.4533
Split [3/5], Epoch [4/5], Val Loss: 2.5000, Accuracy: 0.4519
Split [3/5], Epoch [5/5], Val Loss: 2.4996, Accuracy: 0.4525
Split [4/5], Epoch [1/5], Val Loss: 2.4992, Accuracy: 0.4542
Split [4/5], Epoch [2/5]

In [262]:
test_dataset = CustomDataset(test['tensorized_headline'], test['is_sarcastic'])
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
print(len(test_dataset))
model_lr.eval()

total_correct = 0
total_samples = 0

for inputs, targets in test_dataloader:
  outputs = model_lr(inputs)
  target_one_hot = F.one_hot(targets, 2)
  _, predicted = torch.max(outputs, dim=1)
  total_correct += (predicted == target_one_hot).sum().item()
  total_samples += targets.size(0)

accuracy = total_correct / total_samples
print("Accuracy on test dataset: {:.2%}".format(accuracy))

12614
Accuracy on test dataset: 45.76%
